<a href="https://colab.research.google.com/github/choki0715/UnLiteFlowNet-PIV/blob/master/(UGW_auto_edit)DLinear_TT_%EC%88%98%EC%A7%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<span style="color:#ffd33d; font-size:150%"> 2022.11.07: DLinear Model Demo </span>

> 2022.12.08> UGW : TT water quelity : 29파일

> 2022.12.09> UGW : TT water quelity : codex 수정 3개 파일, 재정제 3개파일

1) encode : 3개

2) 재정제 : 3개


In [1]:
from google.colab import drive
drive.mount('/content/drive')

# 이동 복사한 모델 폴더의 경로를 입력하여 이동시킴
%cd /content/drive/MyDrive/NIA 2차 수질예측 프로젝트/model/UGW/DLinear

Mounted at /content/drive
/content/drive/MyDrive/NIA 2차 수질예측 프로젝트/model/UGW/DLinear


In [2]:
# import argparse
import os
import os.path as osp
import io
import shutil

import logging
import traceback

from datetime import date, datetime
from dataclasses import dataclass, field
from typing import Set, List, Dict, Tuple #, final
from openpyxl import Workbook, load_workbook

import numpy as np
import pandas as pd
import random

import torch
from scipy.stats import pearsonr
from sklearn.metrics import *
from UGW_exp.exp_main_ugw import Exp_Main

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as fm
import seaborn as sns
import IPython
import IPython.display

mpl.rcParams['figure.figsize'] = (16,12)
mpl.rcParams['axes.grid'] = False

from pickle import FALSE
# %matplotlib inline

fix_seed = 2022
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [4]:

@dataclass
class args:
    is_training : bool   # or int
    model_id : str
    model : str

    # data loader
    data : str
    root_path : str
    data_path : str
    save_path : str
    features : str
    target : str
    freq : str
    checkpoints : str

    # forecasting task
    seq_len : int         #56
    label_len : int       #1
    pred_len : int        #5
    # DLinear
    moving_avg : int      #10
    # moving_avg = [3,5,10,15,20,25,30] #,35,40,45]
    conv_kernal : int      # 1
    conv1d : bool          # True
    RIN : bool             # True
    combination : bool     # True
    embed_type : int       # 0
    enc_in : int           # 3
    c_out : int       # 1
    embed : str       # 'timeF'
    do_predict : bool # False

    # optimization
    num_workers : int          # 10
    itr : int                  # 1
    train_epochs : int         # 100
    batch_size : int           # 5
    patience : int             # 7
    learning_rate : float      # 0.01
    des : str                  # 'Exp'
    loss : str                 # 'mse'
    lradj : str                # 'type1'
    use_amp : bool             # False

    # GPU
    use_gpu : bool             # True
    gpu : int                  # 0
    use_multi_gpu : bool       # False
    devices : str              # '0,1,2,3'
    test_flop : bool           # False

@dataclass(frozen=True)
class TS_DATA:
    # base info
    Id : int
    ob_num : str
    ob_id : str
    ob_name : str
    Modi_Date : date
    FileName : str
    FilePath : str
    Row_count : int
    MAE : float
    MSE : float
    RMSE : float
    norm_MAPE : float
    MAPE : float
    ColumnNames : List[str] = field(default_factory=List)
    Row_NA_count : List[int] = field(default_factory=List)
    plot : bool = False
# from openpyxl.styles.fonts import Font

class TSFAR:
    def __init__(self, file_path, save_path, file_format, fxls_name, sheet_name, sheet_col) -> None:
        self.file_path = file_path
        self.save_path = save_path
        self.file_format = file_format
        self.file_list = [f"{file}" for file in os.listdir(self.file_path) if self.file_format in file]
        self.file_list_len = len(self.file_list)
        print(f" --- Find Files for load : {self.file_list_len}\n")
        self.TS_DATA_ANLYS_LIST = []
        # self.MODEL_args_LIST = []
        self.fxls_name = fxls_name
        self.sheet_name = sheet_name
        # self.sheet_col = sheet_col
        self.sheet_col = ['순번','관측정번호','관측망구분','관측망이름','작업일','파일명','파일path','파일행수','MAE','MSE','RMSE','norm_MAPE','MAPE']

        self.TSDATA = TS_DATA

        logging.basicConfig(filename= self.save_path+'/test.log', level=logging.ERROR)

    def TS_DATA_ANLYS_FILES(self):

        TSDATA = self.TSDATA

        if self.fxls_name in os.listdir(self.file_path):
            wb = load_workbook(filename = os.path.join(self.file_path+self.fxls_name))
        else:
            wb = Workbook()
        write_ws = wb.create_sheet(self.sheet_name,-1)
        fig_ws = wb.create_sheet(self.sheet_name+'_plot',-1)
        write_ws = wb.active
        # write_ws.append(self.sheet_col)
        colname = ['순번','관측정번호','관측망구분','관측망이름','작업일','파일명','파일path','파일행수','MAE','MSE','RMSE','norm_MAPE','MAPE']
        colname.extend([])
        write_ws.append(colname)

        for file_ix in range(self.file_list_len):
            print(f"\n=================================================================>> : ")
            print(f"Total Task file Count.... : {self.file_list_len}")
            print(f"Processing.... Now ...... : {file_ix + 1}")
            print(f"=================================================================>> : \n")

            TSDATA = TS_ANLYS(self.file_path, self.file_format, file_ix).Set_TS_Data()

            row_max = write_ws.max_row + 1
            col_max = write_ws.max_column # need sheet col lenth check
            write_ws.cell(row_max,1,TSDATA.Id)
            write_ws.cell(row_max,2,TSDATA.ob_num)
            write_ws.cell(row_max,3,TSDATA.ob_id)
            write_ws.cell(row_max,4,TSDATA.ob_name)
            write_ws.cell(row_max,5,TSDATA.Modi_Date)
            write_ws.cell(row_max,6,TSDATA.FileName)
            write_ws.cell(row_max,7,TSDATA.FilePath)
            write_ws.cell(row_max,8,TSDATA.Row_count)
            write_ws.cell(row_max,8+1,TSDATA.MAE)
            write_ws.cell(row_max,8+2,TSDATA.MSE)
            write_ws.cell(row_max,8+3,TSDATA.RMSE)
            write_ws.cell(row_max,8+4,TSDATA.norm_MAPE)
            write_ws.cell(row_max,8+5,TSDATA.MAPE)
            wb.save(os.path.join(self.save_path,self.fxls_name))

        return TSDATA


class TS_ANLYS:
    def __init__(self,file_path, file_format, file_ix) -> None:
        self.LOAD_TS_FILE = LOAD_TS_FILE(file_path, file_format, file_ix)
        self.file_ix = file_ix
        self.Row_count, self.ColumnNames, self.Row_NA_count, self.columns_NA, self.ob_num,self.ob_id, self.ob_name, self.MAE, self.MSE, self.RMSE, self.norm_MAPE, self.MAPE = self.ANLYS()

    def __repr__(self) -> str:
        return "TS_ANLYS is called"

    def ANLYS(self):
        tsFile_name = self.LOAD_TS_FILE.fileName
        tsFile = self.LOAD_TS_FILE.File_loader()
        args = self.Set_args()
        Exp = Exp_Main

        args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False
        if args.use_gpu and args.use_multi_gpu:
            args.dvices = args.devices.replace(' ', '')
            device_ids = args.devices.split(',')
            args.device_ids = [int(id_) for id_ in device_ids]
            args.gpu = args.device_ids[0]

        # OBS_Point = str
        Row_count = tsFile.shape[0]
        file_info = tsFile_name.rstrip('.csv').split('_')
        ob_num = tsFile.iloc[2,0] # tsFile['obsv_no'][2]
        ob_id = file_info[0]
        ob_name = file_info[1]

        Row_NA_count = tsFile.isna().sum().to_list()
        # print(f"Row_NA_count:\n{type(Row_NA_count)}")
        ColumnNames = tsFile.columns.to_list()
        # anlysis info
        columns_NA : Tuple[str, int] = field(default_factory=Tuple)
        # plot : bool = True

        if args.is_training:
          for ii in range(args.itr):
              # setting record of experiments
              setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_eb{}_{}_{}'.format(
                                                                    args.model_id, args.model, args.data, args.features,
                                                                    args.seq_len, args.label_len, args.pred_len, args.embed,
                                                                    args.des, ii)
              exp = Exp(args)  # set experiments
              print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
              exp.train(setting)

              print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
              preds, trues, inputx = exp.test(setting)
              MAE, MSE, RMSE, norm_MAPE, MAPE = self.Matric(preds, trues)

              if args.do_predict:
                  print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                  exp.predict(setting, True)
              torch.cuda.empty_cache()
        else:
          ii = 0
          setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_eb{}_{}_{}'.format(
                                                                    args.model_id, args.model, args.data, args.features,
                                                                    args.seq_len, args.label_len, args.pred_len, args.embed,
                                                                    args.des, ii)
          exp = Exp(args)  # set experiments
          print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
          preds, trues, inputx = exp.test(setting, test=1)
          torch.cuda.empty_cache()
          MAE, MSE, RMSE, norm_MAPE, MAPE = self.Matric(preds, trues)

        return Row_count, ColumnNames, Row_NA_count, columns_NA, ob_num, ob_id, ob_name, MAE, MSE, RMSE, norm_MAPE, MAPE

    def Set_args(self):
        tsFile_name = self.LOAD_TS_FILE.fileName
        tsFile = self.LOAD_TS_FILE.File_loader()
        file_info = tsFile_name.rstrip('.csv').split('_')
        ob_num = tsFile.iloc[2,0]
        ob_id = file_info[0]
        ob_name = file_info[1]


        model_args = args(
            is_training = True,
            model_id = '(UGW)_TT_water_quality_' + str(ob_num) + '_' + str(ob_name),
            model = 'DLinear',
            # data loader
            data = 'UGW_water_quality',
            root_path = self.LOAD_TS_FILE.file_path,  #'/content/drive/MyDrive/NIA 2차 수질예측 프로젝트/NIA_AIDEN_06_(지하수)/ AI_데이터_null변환(20220914)/국가수위'
            data_path = self.LOAD_TS_FILE.fileName,  # '국가_수위_11742.csv'
            save_path = '/content/drive/MyDrive/NIA 2차 수질예측 프로젝트/NIA_AIDEN_07_(지하수)/수질(Total)/plot/',
            features = 'MS',
            target = 'ec',
            freq = 'h',
            checkpoints = './(UGW_수질)TT_checkpoints/',
            # forecasting task
            seq_len = 72,
            label_len = 1,
            pred_len = 24,
            # DLinear
            moving_avg = 5,
            # args.moving_avg = [3,5,10,15,20,25,30,35,40,45]    # 'window size' list of moving average'
            conv_kernal = 1,
            conv1d = True,
            RIN = True,
            combination = True,
            embed_type = 0,
            enc_in = 5,
            c_out = 1,
            # self.args.factor = 1
            # self.args.distil = True
            embed = 'timeF',
            # self.args.activation = 'relu'
            # self.args.output_attention = True
            do_predict = False,
            # optimization
            num_workers =10,
            itr = 1,
            train_epochs = 50,
            batch_size = 20,
            patience = 5,
            learning_rate = 0.1,
            des = 'Exp',
            loss = 'mse',
            lradj = 'type1',
            use_amp = False ,
            # GPU
            use_gpu = True,
            gpu = 0,
            use_multi_gpu = False,
            devices = '0,1,2,3',
            test_flop = False,
        )
        return model_args

    def Set_TS_Data(self):
        TSDATA = TS_DATA(       Id = self.file_ix+1,
                                ob_num = self.ob_num,
                                ob_id = self.ob_id,
                                ob_name = self.ob_name,
                                Modi_Date = datetime.today(),
                                FileName = self.LOAD_TS_FILE.fileName,
                                FilePath = self.LOAD_TS_FILE.file_path,
                                Row_count = self.Row_count,
                                MAE = self.MAE ,
                                MSE = self.MSE,
                                RMSE = self.RMSE,
                                norm_MAPE = self.norm_MAPE,
                                MAPE = self.MAPE,
                                ColumnNames = self.ColumnNames,
                                Row_NA_count = self.Row_NA_count,
                                plot = True
                                )
        return TSDATA


    def Matric(self,pred, true_y):
        args = self.Set_args()
        t_df_rs = self.LOAD_TS_FILE.File_loader().copy()
        # t_df_rs = self.tsFile.copy()
        t_df_rs['date'] = pd.to_datetime(t_df_rs['obsrvn_ymdh'])
        t_df_rs = t_df_rs.set_index('date')
        t_df_rs = t_df_rs.resample(rule = 'D', kind='timestamp', origin='start').first()
        print(f"t_df_rs.shape : {t_df_rs.shape}")

        num_train = int(len(t_df_rs) * 0.8)
        num_test = int(len(t_df_rs) * 0.1)
        num_vali = len(t_df_rs) - num_train - num_test
        border1 = [0, num_train - args.seq_len, len(t_df_rs) - num_test - args.seq_len]
        border2 = [num_train, num_train + num_vali, len(t_df_rs)]

        train_data = t_df_rs[border1[0]:border2[0]]

        true_step = t_df_rs[border1[2]:border2[2]]
        train_step = train_data
        self.train_mean = train_step.mean()
        self.train_std = train_step.std() + 0.00000001

        #prepare
        result = pred[:].squeeze()
        label = true_y[:].squeeze()

        result_scale = (result*self.train_std[args.target]) + self.train_mean[args.target]
        label_scale = (label*self.train_std[args.target]) + self.train_mean[args.target]

        bk = []
        # bk = [ i for i in range(410,434)]
        # bk.extend([469,470,475,476,477,478,479,480,])
        diff = 0
        # diff = diff + len(bk)

        #corr
        sumout = 0
        for id in range(result.shape[0]-diff):
          if id in bk:
            pass
          else:
            out,_ = pearsonr(label[id,:],result[id,:])
            sumout += out
        corr =  sumout/(result.shape[0]-diff)
        print(f"corr : {corr}")

        #MAE
        # MAE = mean_absolute_error(label, result)
        sumout = 0
        for id in range(result.shape[0]-diff):
          if id in bk:
            pass
          else:
            out = mean_absolute_error(label[id,:],result[id,:])
            sumout += out
        MAE = sumout/(result.shape[0]-diff)
        print(f"MAE : {MAE}")

        #MSE
        sumout = 0
        for id in range(result.shape[0]-diff):
          if id in bk:
            pass
          else:
            out = mean_squared_error(label[id,:],result[id,:])
            sumout += out
        MSE =  sumout/(result.shape[0]-diff)
        print(f"MSE : {MSE}")

        # RMSE (Root Mean Squared Error)
        sumout = 0
        for id in range(result.shape[0]-diff):
          if id in bk:
            pass
          else:
            out = mean_squared_error(label[id,:],result[id,:])
            sumout += np.sqrt(out)
        RMSE =  sumout/(result.shape[0]-diff)
        print(f"RMSE : {RMSE}")

        #MAPE (Mean Absolute Percentage Error)
        sumout = 0
        for id in range(result.shape[0]-diff):
          if id in bk:
            pass
          else:
            out = np.mean(np.abs((label[id,:] - result[id,:]) / label[id,:])) * 100
            sumout += out
        norm_MAPE = sumout/(result.shape[0]-diff)
        # print(f"MAPE : {MAPE}")

        #MAPE_scale (Mean Absolute Percentage Error)
        sumout = 0
        for id in range(result_scale.shape[0]-diff):
          if id in bk:
            pass
          else:
            out = np.mean(np.abs((label_scale[id,:] - result_scale[id,:]) / label_scale[id,:])) * 100
            sumout += out
        MAPE = sumout/(result_scale.shape[0]-diff)
        print(f"denorm_MAPE : {MAPE}")
        # print(f"error-count : {diff}")
        return MAE, MSE, RMSE, norm_MAPE, MAPE


class LOAD_TS_FILE:
    def __init__(self, file_path, file_format, file_ix) -> None:
        self.file_ix = file_ix
        self.file_path = file_path
        self.file_format = file_format
        self.file_list = [f"{file}" for file in os.listdir(self.file_path) if self.file_format in file]
        self.fileName = self.file_list[self.file_ix]

    def File_loader(self):
        if self.file_format == ".csv":
            tsFile = pd.read_csv(os.path.join(self.file_path+self.fileName), header=0)
            print("fime_name : ",self.fileName ," file's shape : ", tsFile.shape)
        # else:
        #     self.file_format == '.xlsx'
        #     tsFile = pd.read_excel(os.path.join(self.file_path+self.fileName), header=0)
        #     print("fime_name : ",self.fileName ," file's shape : ", tsFile.shape)
        return tsFile


In [5]:

## 코드 실행을 위한 쉘.

# 데이터가 있는 위치 폴더를 경로 입력
file_path = "/content/drive/MyDrive/NIA 2차 수질예측 프로젝트/NIA_AIDEN_07_(지하수)/수질(Total)/추가_수질_데이터(1209)/2. 수질 데이터셋(1209)_재정제/"
# 결과 파일이 출력될 폴더 경로 입력
save_path = "/content/drive/MyDrive/NIA 2차 수질예측 프로젝트/NIA_AIDEN_07_(지하수)/수질(Total)/"
# 읽어올 파일 format 조작하지 않아도 됨.
file_format = ".csv"
# 출력될 엑셀 파일 이름 지정
fxls_name =  "UGW_DLinear_TT_water_quality_1209_(data_fix)_02.xlsx"
# 출력될 엑셀 파일의 sheet명
sheet_name = "TT_water_quality"
sheet_col = ""

autoAR = TSFAR(file_path, save_path, file_format, fxls_name, sheet_name, sheet_col)
autoAR.TS_DATA_ANLYS_FILES()

 --- Find Files for load : 1


=================================================================>> : 
Total Task file Count.... : 1
Processing.... Now ...... : 1
=================================================================>> : 

fime_name :  보조_양수_암반_수질_시.csv  file's shape :  (31767, 36)
fime_name :  보조_양수_암반_수질_시.csv  file's shape :  (31767, 36)
Use CPU
>>>>>>>start training : (UGW)_TT_water_quality_2_양수_DLinear_UGW_water_quality_ftMS_sl72_ll1_pl24_ebtimeF_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
df_raw.cols : Index(['Unnamed: 0', 'well_num', 'obsrvn_ymdh', 'ec', 'wt_temp', 'rainfall',
       'wd_temp', 'lev', 'no3n', 'bac', 'cl', 'cd', 'gas', 'cn', 'hg', 'diazn',
       'parat', 'phenol', 'pb', 'cr', 'tcet', 'tece', 'tce', 'benzene',
       'toluene', 'etilben', 'xylene', 'CA2', 'Mg2', 'Na', 'K', 'NO3', 'SO42',
       'CL', 'CO32', 'HCO3'],
      dtype='object')
df_data.cols : Index(['Unnamed: 0', 'wt_temp', 'rainfall', 'wd_temp', 'ec'], dtype='object')
df_da

TS_DATA(Id=1, ob_num=2, ob_id='보조', ob_name='양수', Modi_Date=datetime.datetime(2023, 11, 12, 6, 33, 58, 72007), FileName='보조_양수_암반_수질_시.csv', FilePath='/content/drive/MyDrive/NIA 2차 수질예측 프로젝트/NIA_AIDEN_07_(지하수)/수질(Total)/추가_수질_데이터(1209)/2. 수질 데이터셋(1209)_재정제/', Row_count=31767, MAE=0.11298173245357329, MSE=0.045581324236461766, RMSE=0.1325189904820702, norm_MAPE=66.90074673444877, MAPE=8.201914530419478, ColumnNames=['Unnamed: 0', 'well_num', 'obsrvn_ymdh', 'ec', 'wt_temp', 'rainfall', 'wd_temp', 'lev', 'no3n', 'bac', 'cl', 'cd', 'gas', 'cn', 'hg', 'diazn', 'parat', 'phenol', 'pb', 'cr', 'tcet', 'tece', 'tce', 'benzene', 'toluene', 'etilben', 'xylene', 'CA2', 'Mg2', 'Na', 'K', 'NO3', 'SO42', 'CL', 'CO32', 'HCO3'], Row_NA_count=[0, 0, 0, 0, 0, 0, 0, 0, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31767, 31